In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/daiv_dataset_final_final_preprocessed.csv', encoding='UTF-8')

# StandardScaler 사용
scaler = StandardScaler()

y = data.iloc[:, [6]]
X = data.drop(columns=data.columns[:8])

# StandardScaler 사용
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X

,gender,heparin,clopidogrel,insulin,digoxin,sprionolactone,bumetanide,torsemide,furosemide,metolazone,...,magnesium_max,magnesium_min,hematocrit_min,hemoglobin_max,hematocrit_max,platelet_count_mean,urea_nitrogen_mean,platelet_count_min,hematocrit_mean,race_encoded
0,0,0,0,0,0,0,0,0,0,0,...,2.172606,1.804138,31.741238,11.858272,35.729242,240.294055,23.691527,213.197345,33.611756,0.168196
1,0,1,0,1,0,0,0,0,1,0,...,1.800000,1.700000,28.500000,11.500000,35.400000,190.000000,16.333300,176.000000,32.680000,0.587577
2,1,1,0,1,0,0,0,1,0,0,...,1.918279,1.918279,39.300000,12.900000,39.300000,95.000000,43.500000,95.000000,39.300000,0.587577
3,1,1,0,1,0,0,0,0,0,0,...,2.100000,1.900000,30.400000,12.300000,37.700000,571.000000,14.400000,535.000000,32.928600,0.587577
4,1,1,0,1,0,0,0,0,0,0,...,2.200000,2.000000,35.300000,12.600000,37.400000,317.000000,15.500000,316.000000,36.350000,0.587577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19395,0,1,0,1,0,0,0,0,0,0,...,2.172606,1.804138,31.741238,11.858272,35.729242,240.294055,23.691527,213.197345,33.611756,0.587577
19396,0,1,0,1,0,0,0,0,0,0,...,2.172606,1.804138,31.741238,11.858272,35.729242,240.294055,23.691527,213.197345,33.611756,0.587577
19397,0,1,0,1,0,0,0,0,0,0,...,2.172606,1.804138,31.741238,11.858272,35.729242,240.294055,23.691527,213.197345,33.611756,0.587577
19398,0,1,0,1,0,0,0,0,0,0,...,2.172606,1.804138,31.741238,11.858272,35.729242,240.294055,23.691527,213.197345,33.611756,0.587577


In [ ]:
# cad_icd_code 열 처리
selected_column_cad = y['cad_icd_code']
binary_labels_cad = selected_column_cad.notna().astype(int)
y['binary_cad_icd_code'] = binary_labels_cad


# 원본 열 삭제
y = y.drop(columns=['cad_icd_code'])

<ipython-input-4-972a5516ad81>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['binary_cad_icd_code'] = binary_labels_cad


In [ ]:
y

,binary_cad_icd_code
0,0
1,1
2,1
3,1
4,0
...,...
19395,1
19396,1
19397,1
19398,1


In [ ]:
# 데이터를 8:2 비율로 훈련 및 테스트 세트로 분할하고 셔플링
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=True)

**XGBoost**

In [ ]:
import xgboost as xgb

# XGBoost 모델 초기화
xgb_model = xgb.XGBClassifier(random_state=0, n_estimators=400, scale_pos_weight = 0.1)

# 훈련 데이터에 모델 훈련
xgb_model.fit(train_X, train_y)

# 테스트 데이터로 예측
y_pred = xgb_model.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
print("Accuracy:", accuracy)

y_pred_proba_xgb = xgb_model.predict_proba(test_X)[:, 1]
roc_auc = roc_auc_score(test_y, y_pred_proba_xgb)
print("AUC ROC:", roc_auc)

Accuracy: 0.8458762886597938
AUC ROC: 0.7424344322344323


XGBoost with Resample module

In [ ]:
import numpy as np
from sklearn.utils import resample
from xgboost import XGBClassifier

In [ ]:
# 다수 클래스와 소수 클래스로 분리
majority_class = train_X[train_y == 0]
minority_class = train_X[train_y == 1]

# 소수 클래스를 다수 클래스와 동일한 수로 언더샘플링
majority_class_downsampled = resample(majority_class, replace=True, n_samples=len(minority_class), random_state=0)

# 언더샘플링된 다수 클래스와 소수 클래스를 결합
undersampled_X = pd.concat([majority_class_downsampled, minority_class])
undersampled_y = np.concatenate([np.zeros(len(minority_class)), np.ones(len(minority_class))])

In [ ]:
# XGBoost 모델 초기화 및 학습
xgb_model = XGBClassifier(random_state=0)
xgb_model.fit(undersampled_X, undersampled_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=0, ...)

In [ ]:
predictions = xgb_model.predict(test_X)

accuracy = accuracy_score(test_y, predictions)
print("Accuracy:", accuracy)

probs = xgb_model.predict_proba(test_X)[:, 1]
auc = roc_auc_score(test_y, probs)
print("AUC:", auc)

Accuracy: 0.8376288659793815
AUC: 0.5


XGBoost with RUS

In [ ]:
import xgboost as xgb
from imblearn.under_sampling import RandomUnderSampler

# 언더샘플링 수행
rus = RandomUnderSampler(random_state=0)
train_X_resampled, train_y_resampled = rus.fit_resample(train_X, train_y)

# XGBoost 모델 초기화
xgb_model = xgb.XGBClassifier(random_state=0)

# 훈련 데이터에 모델 훈련
xgb_model.fit(train_X_resampled, train_y_resampled)

# 테스트 데이터로 예측
y_pred = xgb_model.predict(test_X)

accuracy = accuracy_score(test_y, y_pred)
print("Accuracy:", accuracy)

y_pred_proba_xgb = xgb_model.predict_proba(test_X)[:, 1]
roc_auc = roc_auc_score(test_y, y_pred_proba_xgb)
print("AUC ROC:", roc_auc)

Accuracy: 0.6961340206185567
AUC ROC: 0.7622840048840049
